# Manga H5 Creation

File used to create H5 dataset for manga dataset

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import h5py

from astropy.io import fits
from glob import glob


from importlib import reload
import Modeling
reload(Modeling)
import Modeling
from Modeling import CrossValidationModeler, EbossWaveSpectraDataset, MangaWaveSpectraDataset

from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Convolution1D, Dropout, MaxPooling1D, Conv2D, Conv1D, InputLayer, Dropout
from keras.optimizers import SGD
from keras.utils import np_utils

Seperate Needed values from the dataset.

In [38]:
manga_data = pd.read_csv('../../Data/Astronomy20000_Original.csv')
# Extract the duplicates
excat_duplicates = manga_data[manga_data.duplicated(keep='last')]
id_duplicates_excluding_excat = manga_data[~manga_data.duplicated(keep=False) & manga_data.duplicated(subset='id', keep=False)]
# Drop them.
manga_data.drop(excat_duplicates.index, inplace=True)
manga_data.drop(id_duplicates_excluding_excat.index, inplace=True)
manga_ids = manga_data[['id', 'Hits']]

# Place fitler here.
manga_ids = manga_ids[manga_ids.id.str.contains('multiline')]

# Collectors for files
not_found_files_general = []
found_files_general = []

# Creating variables needed for file existance checking.
file_name_template = 'C:/Users/stone/Desktop/Summer2018/AstroProj/Data/RawManga\HIT_RSS_mpl5_{plate_id}_{mjd}_fiber_{fiber_id}_{type}.fits'
file_list = set(glob('C:/Users/stone/Desktop/Summer2018/AstroProj/Data/RawManga/*.fits'))

# For each id in the initial dataset parse it and read in the values.
for _, e_id, hit in manga_ids.itertuples():
    plate_id, mjd, _, fiber_id, file_type = e_id.split('_')
    file_to_read = file_name_template.format(plate_id=plate_id, mjd=mjd, fiber_id=fiber_id, type=file_type)
    if file_to_read in file_list:
        fits.open(file_to_read)
        found_files_general.append((e_id, file_to_read, file_type, hit))
    else:
        not_found_files_general.append(e_id)
        
manga_locations = pd.DataFrame(found_files_general, columns=['id', 'location', 'file_type', 'hit'])
manga_locations.to_csv('manga_locations_multiline.csv')
len(not_found_files_general), len(found_files_general)

(0, 2184)

Generate the h5py dataset.

In [12]:
del file['flux_values']
del file['flux_labels']
del file['flux_ids']
del file['res_flux_values']
del file['ivar_flux_values']

In [13]:
file.close()

In [16]:
mwsd = MangaWaveSpectraDataset('CSVS/manga_locations_oneline.csv')
num_of_spectra = len(mwsd)

#del file['flux_values']
#del file['flux_labels']
#del file['flux_ids']

#file.close()

#H5 File setup
file = h5py.File('../../Data/manga_flux_oneline+.hdf5', mode='w')
file.create_dataset('flux_values', shape=(num_of_spectra, 4563))
file.create_dataset('res_flux_values', shape=(num_of_spectra, 4563))
file.create_dataset('ivar_flux_values', shape=(num_of_spectra, 4563))
file.create_dataset('ivar_rescaled_flux_values', shape=(num_of_spectra, 4563))
file.create_dataset('flux_labels', shape=(num_of_spectra, 1))
file.create_dataset('flux_ids', shape=(num_of_spectra, 1), dtype='S33')

# Place this in the for loop.
label_mapper = lambda x: 0 if x == 'bad' else 1
mwsd.spectra_infos.id = mwsd.spectra_infos.id.astype('S')
for ix in range(len(mwsd)):
    row = mwsd[ix]
    file['flux_values'][ix] = row[0]
    file['flux_labels'][ix] = label_mapper(row[1].hit)
    file['flux_ids'][ix] = row[1].id
    
    file['ivar_flux_values'][ix] = mwsd.get_ext_data(ix, 4)[0]
    file['ivar_rescaled_flux_values'][ix] = mwsd.get_ext_data(ix, 5)[0]
    file['res_flux_values'][ix] = mwsd.get_ext_data(ix, 7)[0]
    
file.close()